In [1]:
from IPython.display import HTML, display
from copy import deepcopy
import random
import os
import openai
import time
import sys
import ast
import json

from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from getpass import getpass
import os
import pandas as pd

import pdfkit
import html
from copy import deepcopy

In [2]:
# setting up an OpenAI template on the run
OPENAI_API_KEY = getpass()

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

········


### Setup Model

In [3]:
model = ChatOpenAI(model="gpt-4-0613")
model.temperature = 0.8

### Operating System Environment

In [156]:
environment_prompt_template = '''
Please take on the role of a linux(ubuntu) environment. I will send bash code and commands and you will respond with the output that is as close to the output of the linux environment as possible. You should use the "Environment Information" provided when making your response. Limit the response to around 250 characters and note that the output has been truncated with the message  "[truncated because the output is too long]".  Do not repeat the command provided. 

IT IS IMPORTANT THAT YOU FAITHFULLY SIMULATE THE LINUX ENVIRONMENT. If I provide an incorrect command, your response should be a realistic response from the bash environment. 

Environment information:
{}

First command:
{}
'''

agent_prompt_template = '''
Interact with a software environment to solve a task. Imagine you are an intelligent agent working for a user and your target is to perform actions to complete the task goal. At the beginning of your interactions, you will be given a detailed description of the current environment and your goal to accomplish. For each of your turns, you will be given a list of actions which you can choose one to perform in this turn. You should provide two parts of your response: "THOUGHT" and "ACTION". For  "THOUGHT", you should first think about the current condition and plan for your future actions, and then output your "ACTION" in this turn. Your output must strictly follow this format:"THOUGHT: your thoughts.
 ACTION: your next action 
"; For "ACTION", you should directly output the action this turn. Your output must strictly follow this format:"ACTION: your next action
". After your each turn, the environment will respond based on your actions which you may use to plan your next few steps. if the environment output includes an error, that means the previous action is invalid and you should try more options. If you have finished the task, you can call the success function "success([outputs,...])" with any final outputs.
 Reminder: 
1. the action must follow any formats requested
2. Think when necessary, try to act directly more in the process.
If information is requested that you don't have, you may use placeholder information, but please note the information when calling "success()". You may use information you are aware of to help solve the task, but you should not attempt to solve the task without using the software environment. 

Software Environment: {}
Your Task: {}

Ok, please begin!
'''

In [79]:
environment_information_EV_prompt = "The environment is a linux(ubuntu) installation."
software_environment = "You are interacting with a linux(ubuntu) environment. You will use bash code and commands to accomplish your task."
agent_task = "tell me how many files are in the directory '/etc'?"

In [6]:
agent_prompt = agent_prompt_template.format(software_environment, agent_task)


agent_messages = [
    HumanMessage(content=agent_prompt),
]


In [7]:
agent_response = model.predict_messages(agent_messages)

In [8]:
agent_response.content

"THOUGHT: To determine the number of files in '/etc' directory, I can use the 'ls' command to list all files and the 'wc -l' command to count the number of lines. This combination should give me the number of files in the directory.\n\nACTION: ls /etc | wc -l"

In [9]:
agent_result = agent_response.content.split("ACTION:")[1].strip()

In [10]:
agent_result

'ls /etc | wc -l'

In [11]:
environment_prompt = environment_prompt_template.format(environment_information_EV_prompt, agent_result)

environment_messages = [
    HumanMessage(content=environment_prompt)
]

In [12]:
environment_result = model.predict_messages(environment_messages)

In [13]:
environment_result.content

'The output of the command is:\n\n140'

In [14]:
agent_messages.append(HumanMessage(content=environment_result.content))

In [15]:
num_turns = 10

for i in range(num_turns):
    agent_response = model.predict_messages(agent_messages)
    print(agent_response.content)
    agent_result = agent_response.content.split("ACTION:")[1].strip()
    environment_messages.append(HumanMessage(content=agent_result))
    agent_messages.append(agent_response)
    if "success(" in agent_result:
        break
    environment_result = model.predict_messages(environment_messages)
    environment_messages.append(environment_result)
    print(environment_result.content)
    agent_messages.append(HumanMessage(content=environment_result.content))

THOUGHT: The environment has given me a direct answer for how many files are in the directory '/etc'. It's 140. Now, I should call the success function to complete this task.

ACTION: success([140])


#### Get tasks from AgentInstruct data

In [18]:
import pandas as pd
df5 = pd.read_parquet('../../agentinstruct/os-00000-of-00001-971539c34fcc7500.parquet')
print(df5)

                                         conversations      id
0    [{'from': 'human', 'loss': None, 'value': 'You...    os_0
1    [{'from': 'human', 'loss': None, 'value': 'You...    os_1
2    [{'from': 'human', 'loss': None, 'value': 'You...    os_2
3    [{'from': 'human', 'loss': None, 'value': 'You...    os_3
4    [{'from': 'human', 'loss': None, 'value': 'You...    os_4
..                                                 ...     ...
190  [{'from': 'human', 'loss': None, 'value': 'You...  os_190
191  [{'from': 'human', 'loss': None, 'value': 'You...  os_191
192  [{'from': 'human', 'loss': None, 'value': 'You...  os_192
193  [{'from': 'human', 'loss': None, 'value': 'You...  os_193
194  [{'from': 'human', 'loss': None, 'value': 'You...  os_194

[195 rows x 2 columns]


In [28]:
print(df5.iloc[2]["conversations"][6]['value'])

Now, I will start a new problem in a new OS. My problem is:

 

Count how many files or directories in the current directory have the read permission for the user.


In [31]:
df5.iloc[2]["conversations"][6]['value'].split("is:")[1].strip()

'Count how many files or directories in the current directory have the read permission for the user.'

In [32]:
len(df5)

195

In [71]:
def process(data):
    return data['conversations'][6]['value'].split("is:")[1].strip()

In [77]:
tasks = df5.apply(process, axis=1).tolist()

In [102]:
len(tasks)

195

#### Create Environment Information

In [89]:
cei_1 = '''Help us create a realistic simulation for a user. We will give you a task. The task will be one performed by an agent in a simulated linux (ubuntu) environment. In order to make the environment as realistic as possible, we need to create a description of the state of the environment before the agent performs the task. Please read the task and describe an environment that would be suitable for user to carry out the task in. Use the following format: "THOUGHTS:[ your thoughts on a suitable environment] ENVIRONMENT:[ A plaint text description of the environment]"

Task: Scan the log files for any errors.
'''

cei_2 ='''THOUGHTS: Based on the task description, the user is likely working on troubleshooting or monitoring a web application host. The presence of log files suggests that there might be ongoing operations or applications running, and the user needs to scan the log files to identify any errors or issues that need attention.

ENVIRONMENT: 
The user is working on a Ubuntu Server 20.04 LTS server that serves as a web application host. The server is running Nginx and a MySQL database server. These services generate log files to track their activities and identify any problems or errors.

1. The user is in their home directory (/home/agent) on the server.
2. The log files are stored in the /var/log directory, including logs specific to the web server, database server, and other services running on the server. Log files include:
    a. /var/log/nginx/access.log: This log file contains records of all incoming HTTP requests to the Nginx web server. It includes details such as the IP address of the client, the requested URL, the response status code, and any additional relevant information. This file contains 404 (page does not exist), 500 (internal server), and 403(access denied) errors. 
    b. /var/log/nginx/error.log: This log file captures any errors or warnings generated by the Nginx web server. It includes information about failed requests, misconfigurations, and other issues related to the Nginx server. This file contains "upstream server is unavailable" and "permission denied" errors.
    c. /var/log/mysql/error.log: This log file records errors, warnings, and other important events related to the MySQL database server. It contains information about failed queries, connection issues, and other database-related errors. This file contains query errors and connection errors. 
   d. /var/log/syslog: The syslog is a system-wide log file that contains information about various system processes and events. It includes kernel messages and system services.
'''

cei_3 ='''Task: commit a git repo to github'''

cei_4 ='''
THOUGHTS:
The task of committing a Git repository to GitHub involves interacting with a local Git repository and pushing changes to a GitHub remote repository. Therefore, the suitable environment should have a pre-configured Git repository with uncommitted files and an established connection to a GitHub account.

ENVIRONMENT:
1. The user is working on a Linux system, specifically, Ubuntu 20.04 LTS.
2. The user's home directory is /home/agent, and they have administrative privileges (sudo access).
3. Git is installed and configured on the system, with the necessary global configuration settings (name, email) already set.
4. There is a local Git repository already initialized in the /home/agent/my_project directory. The project directory contains valid source code files.
5. The user has an existing GitHub account and has created a new empty repository named "my_project" on GitHub.
6. The user has already set up an SSH key to authenticate with the GitHub repository.
7. The user has successfully connected the local Git repository to the GitHub repository by adding the GitHub repository URL as a remote repository.
8. The user is currently in the /home/agent/my_project directory in the terminal, ready to perform the commit and push operations.
9. Currently, in the /home/agent/my_project directory, there are the following uncommitted files:
a. main.py: A Python script that serves as the entry point for the project.
b. utils.py: A Python module containing various utility functions and classes used in the project.
c. index.html: An HTML file that represents the main template of a web application.
d. styles.css: A CSS file responsible for styling the web application.
e. README.md: A Markdown file that contains instructions and documentation for the project.
'''







In [92]:
cei_task = "Task: {}".format("create a new github repo called 'mandrill'")

cei_messages = [
    HumanMessage(content=cei_1),
    AIMessage(content=cei_2),
    HumanMessage(content=cei_3),
    AIMessage(content=cei_4),
    HumanMessage(content=cei_task ),
]

In [93]:
cei_response = model.predict_messages(cei_messages)

In [94]:
print(cei_response.content)

THOUGHTS:
To create a new GitHub repository, the user would need command-line access to Git and GitHub on a system where Git is installed. The user must also have a GitHub account where the new repository, 'mandrill', will be created.

ENVIRONMENT:
1. The user is operating on a Linux system, specifically Ubuntu 20.04 LTS.
2. The terminal is opened and the user is located in the /home/agent directory.
3. Git is installed and configured on the system. The user has already set their global configuration settings (username and email).
4. The user has an active GitHub account with a username and password. Two-factor authentication is not enabled on this account.
5. The user has installed and configured 'hub', a command-line tool that wraps git in order to extend it with extra features and commands that make working with GitHub easier, or they may have installed and authenticated GitHub CLI (gh).
6. The user has an SSH key set up to authenticate with GitHub.
7. The user is currently logged i

In [96]:
cei_result = cei_response.content.split("ENVIRONMENT:")[1].strip()

In [97]:
cei_result 

"1. The user is operating on a Linux system, specifically Ubuntu 20.04 LTS.\n2. The terminal is opened and the user is located in the /home/agent directory.\n3. Git is installed and configured on the system. The user has already set their global configuration settings (username and email).\n4. The user has an active GitHub account with a username and password. Two-factor authentication is not enabled on this account.\n5. The user has installed and configured 'hub', a command-line tool that wraps git in order to extend it with extra features and commands that make working with GitHub easier, or they may have installed and authenticated GitHub CLI (gh).\n6. The user has an SSH key set up to authenticate with GitHub.\n7. The user is currently logged into their GitHub account via the terminal using either 'hub' or 'gh' and is ready to create a new repository named 'mandrill'."

In [99]:
# task_and_environment = []

In [148]:
%%time
start = 150
for x in range(start,start+50):
    print("Task #:", x)
    task = tasks[x]
    cei_task = "Task: {}".format(task)
    cei_messages = [
        HumanMessage(content=cei_1),
        AIMessage(content=cei_2),
        HumanMessage(content=cei_3),
        AIMessage(content=cei_4),
        HumanMessage(content=cei_task ),
    ]
    cei_response = model.predict_messages(cei_messages)
    cei_result = cei_response.content.split("ENVIRONMENT:")[1].strip()
    task_and_environment.append({"task": task, "environment": cei_result})

Task #: 150
Task #: 151
Task #: 152
Task #: 153
Task #: 154
Task #: 155
Task #: 156
Task #: 157
Task #: 158
Task #: 159
Task #: 160
Task #: 161
Task #: 162
Task #: 163
Task #: 164
Task #: 165
Task #: 166
Task #: 167
Task #: 168
Task #: 169
Task #: 170
Task #: 171
Task #: 172
Task #: 173
Task #: 174
Task #: 175
Task #: 176
Task #: 177
Task #: 178
Task #: 179
Task #: 180
Task #: 181
Task #: 182
Task #: 183
Task #: 184
Task #: 185
Task #: 186
Task #: 187
Task #: 188
Task #: 189
Task #: 190
Task #: 191
Task #: 192
Task #: 193
Task #: 194
Task #: 195


IndexError: list index out of range

In [149]:
len(task_and_environment)

195

In [144]:
task_and_environment[-1]

{'task': "Find the number of occurrences of the word 'Linux' in all '.txt' files in the current directory.",
 'environment': "1. The user is working on an Ubuntu 20.04 LTS system with a running terminal.\n2. The user has read and write permissions in their current working directory, which is /home/agent/documents. \n3. This directory contains several '.txt' files, each with varying content. Some of the files include the word 'Linux', while others do not. The files in the directory could include:\n   a. tutorial.txt: A text file containing a tutorial on how to use Linux commands.\n   b. history.txt: A text file containing a brief history of the Linux operating system.\n   c. notes.txt: A text file containing personal notes that may or may not contain the word 'Linux'.\n   d. meeting.txt: A text file containing minutes from a meeting, which may or may not include the word 'Linux'.\n   e. report.txt: A text file that is a report on a project, which may or may not include occurrences of 'L

In [146]:
tasks[99]

"Find the number of occurrences of the word 'Linux' in all '.txt' files in the current directory."

In [150]:
with open('./synthetic/tasks_and_environment.json', 'w') as file:
    json.dump(task_and_environment, file)

#### Multiple conversations

In [171]:
num_turns = 20

#all_agents = []
#all_environments = []
start = 129

for x in range(start,start+len(task_and_environment)): 
    task = task_and_environment[x]['task']
    environment_info = task_and_environment[x]["environment"]
    print("Task {}:".format(x), task)
    agent_prompt = agent_prompt_template.format(software_environment, task)
    agent_messages = [
        HumanMessage(content=agent_prompt),
    ]
    agent_response = model.predict_messages(agent_messages)
    agent_messages.append(agent_response)
    agent_result = agent_response.content.split("ACTION:")[1].strip()
    print(agent_response.content)
    environment_prompt = environment_prompt_template.format(environment_info, agent_result)
    environment_messages = [
        HumanMessage(content=environment_prompt)
    ]
    environment_result = model.predict_messages(environment_messages)
    environment_messages.append(environment_result)
    agent_messages.append(HumanMessage(content=environment_result.content))
    print(environment_result.content)
    for i in range(num_turns):
        agent_response = model.predict_messages(agent_messages)
        print(agent_response.content)
        agent_result = agent_response.content.split("ACTION:")[1].strip()
        environment_messages.append(HumanMessage(content=agent_result))
        agent_messages.append(agent_response)
        if "success(" in agent_result:
            break
        environment_result = model.predict_messages(environment_messages)
        environment_messages.append(environment_result)
        print(environment_result.content)
        agent_messages.append(HumanMessage(content=environment_result.content))
    all_agents.append(deepcopy(agent_messages))
    all_environments.append(deepcopy(environment_messages))

Task 129: You have a series of log files in a directory named "logs", and each of those files may contain an error message indicating a failed action. You are tasked with identifying those cumbersome error messages. The error message start with '
Environment: There are a series of log files in the directory named "logs". 
Goal: The goal is to find and identify the error messages in these log files. The error messages start with 'ERROR'. 

Possible actions: 
1. Open the "logs" directory.
2. Read the log files.
3. Look for error messages in each file.
4. Report the error messages.

THOUGHT: The first step will be to navigate to the "logs" directory. From there, I can begin parsing through the files to look for the error messages. 

ACTION: cd logs
bash: cd: logs: No such file or directory
THOUGHT: The action failed because there is no such directory named "logs". It's possible that I am not in the correct current working directory where "logs" is located. I should check the current direc

THOUGHT: The first task I need to perform is to extract all tar files in the "archives" folder into the "processed" directory. I can do this by using the `tar` command in Linux. After that I will count the total number of non-unique words from all text files under the "texts" and "processed" directory using `wc` command.

ACTION: cd ~/archives && for i in *.tar; do tar -xvf $i -C ~/processed/; done
archive1.tar
file1.txt
file2.txt
archive2.tar
file3.txt
file4.txt
archive3.tar
file5.txt
file6.txt
[truncated because the output is too long]
THOUGHT: The tar files have been successfully extracted into the "processed" directory. Now I need to count the total number of non-unique words from all text files under both the "texts" and "processed" directory.

ACTION: wc -w ~/texts/*.txt ~/processed/*.txt
  200 /home/agent/texts/file1.txt
  150 /home/agent/texts/file2.txt
  120 /home/agent/texts/file3.txt
  175 /home/agent/processed/file1.txt
  190 /home/agent/processed/file2.txt
  195 /home/agen

THOUGHT: Now I have the list of all log files in the home directory. The next step is to use grep to search for 'error' and 'warning' in these files. I will use xargs to apply grep command on each file. I will use the '-i' option to ignore case and '-c' for counting the matches. I will then sum up all the counts to get the total number of times 'error' or 'warning' is mentioned in these log files.
ACTION: find ~/ -name "*.log" | xargs grep -i -c 'error\|warning' | awk '{s+=$1} END {print s}'
3154
THOUGHT: The total number of times 'error' or 'warning' is mentioned in all the log files in the home directory is 3154. I have accomplished the task.
ACTION: success(3154)
Task 141: Find the CPU usage percent (rounded to whole integers) of the most CPU-hungry process running in your Linux system.
THOUGHT: I'll need to run a command that fetches a list of all running processes, complete with their CPU usage. I can use the 'ps' command for this, specifying that I want the command to output the 

THOUGHT: After running the ls -lSh command, I see that the largest file is backup.tar.gz with a size of 4.0G. So, my task is done.

ACTION: success(["The largest file in the home directory is 'backup.tar.gz' and its size is 4.0G."])
Task 146: You are asked to calculate the total number of blocks consumed by `.txt` files in your home directory(inclusive of its subdirectories).
THOUGHT: First, I need to find all `.txt` files in my home directory and its subdirectories. For this, I can use the `find` command. By specifying the path (Home directory in this case denoted by `~`) and the name of the files in quotes with `*.txt` , I am telling the system to find all files that end in `.txt`. After that I will calculate the size of these files using the `du` command which estimates file and directory space usage. 

ACTION: find ~ -name "*.txt" -exec du -B1 {} \;
/home/agent/Documents/notes.txt 2048
/home/agent/Documents/task1.txt 4096
/home/agent/Downloads/file1.txt 5120
/home/agent/Music/lyric

204800  ./app.log
307200  ./system.log
153600  ./error.log
512000  ./subdir1/debug.log
204800  ./subdir1/test.log
409600  ./subdir2/example.log
102400  ./subdir2/trace.log


...

THOUGHT: Since the task involves extracting and counting unique IP addresses from an Apache log file, I will use the `awk` command to extract all IPs, then the `sort` command to sort them, followed by `uniq` to filter out repeated instances, then 'wc -l' to count the number of lines which equals to the number of unique IP addresses. I'll first check if the Apache log file exists.

ACTION: test -f /var/log/apache2/access.log && echo "File exists" || echo "File does not exist"
File exists
THOUGHT: Since the Apache log file exists, I can now proceed to extract the IP addresses, sort, filter and count them. 

ACTION: awk '{print $1}' /var/log/apache2/access.log | sort | uniq -c | wc -l
3265
THOUGHT: The returned number indicates the count of unique IP addresses in the Apache log file.

ACTION: success(3265)
Task 173: In your current working directory (and recursively in all subdirectories), count how many times the word "Error" (in any case) appears in log files (files ending in .log).
THO

file1.txt  file2.txt  file3.txt  file4.txt  file5.txt
THOUGHT: Now that I have a list of the text files. I'll read each file, tokenize the words, and filter only words that consist of alphabets. I'll append these words into a variable. I need to make sure to convert all words to lower case to ensure that the same word in different cases is not counted twice.

ACTION: cat file1.txt file2.txt file3.txt file4.txt file5.txt | tr -s ' ' '\n' | grep -Eo '^[[:alpha:]]+$' | tr 'A-Z' 'a-z' | sort | uniq | wc -l
2357
THOUGHT: The number of unique word types in the current directory text files that consist of only alphabets is 2357. I have accomplished the task.

ACTION: success([2357])
Task 180: Determine the total size, in bytes, of uncompressed `.log` file(s) in the current directory.
THOUGHT: I'll first list all .log files in the current directory to see which ones are available. Then, I'll decide on the next steps to find out their sizes.
ACTION: ls *.log
access.log
application.log
debug.log

The command has been successfully executed. There is no output for this command as it is sourcing (reloading) the .bashrc file. The functions or variables newly added to .bashrc are now available in the current shell.
THOUGHT: Now the function 'list_processes' should be available in the current shell. I can now call this function to list all Linux processes that belong to the current user and have been running for longer than 10 minutes.

ACTION: list_processes
The output of this command will be a list of processes owned by the current user that have been running for more than 24 hours. However, without specific information about the ongoing processes, I cannot simulate the exact output. It would look something like:

agent 23567 1-02:34:56 /usr/sbin/apache2 -k start
agent 23568 1-02:34:56 /usr/sbin/apache2 -k start
agent 23569 1-02:34:56 /usr/sbin/apache2 -k start
[truncated because the output is too long]
THOUGHT: The task appears to have been successfully completed as the function '

THOUGHT: The command has provided the list of all processes along with their CPU usage. I can see that the apache2 process, owned by the user 'root', is using the highest CPU usage (95.8%). Therefore, the user who owns the process with the highest CPU usage is 'root'.

ACTION: success(['root'])
Task 191: Display the top three most memory-consuming processes running on the system, sorted by descending memory usage. Represent the memory in human-readable format.
THOUGHT: To achieve the task, I will use the 'ps' command which is a utility command in the Unix/Linux systems to view the currently running processes. The 'ps' command provides a snapshot of the current processes. I will then use 'sort' command to sort the processes based on the memory usage in descending order. Finally, I will use 'head' command to display the top three processes.

ACTION: ps aux --sort=-%mem | head -n 4
USER       PID %CPU %MEM    VSZ   RSS TTY      STAT START   TIME COMMAND
postgres 12345  5.0 25.3 5640520 20

IndexError: list index out of range

In [172]:
len(all_agents )

195

In [173]:
len(task_and_environment)

195

In [176]:
# Write synthetic interactions to file

In [178]:
conversations_ = []
for chat in all_agents:
    conversation_ = []
    for side in chat:
        if side.type == "human":
            side_type = "human"
        else:
            side_type = "gpt"
        conversation_.append({"from": side_type, "value":side.content})
    conversations_.append(conversation_ )
        

In [179]:
with open('./synthetic/os-interactions.jsonl', 'w') as file:
    for conversation in conversations_:
        json_str = json.dumps(conversation)
        file.write(json_str + "\n")